<b> Import Library, find available devices </b>

In [ ]:
# Initialization code for Beta Multiplier Reference Testing.
# This block identifies and initializes the connected SMUs (Source Measure Units).
# If running for the first time, ensure the SMU names are correctly referenced in the subsequent code block.

# This program (ENTER DESCRIPTION)
# This program 
# This program creates a .csv file to save the measurements.

# The setup assumes the use of three SMUs:
# - 
# - 

import pyvisa                    # Importing the pyvisa library for instrument communication
import time                      # Importing the time module to handle time-related tasks (e.g., delays)
import matplotlib.pyplot as plt  # Importing matplotlib.pyplot for plotting graphs and visualizing data
import numpy as np               # Importing numpy for numerical operations, particularly with arrays
import pandas as dp              # Importing pandas as dp for data manipulation and analysis
import os                        # Importing os to interact with the operating system, such as handling file paths
import csv                       # Importing csv to read from and write to CSV files
from datetime import date           # Importing date to read current date
rm = pyvisa.ResourceManager()    # Create a ResourceManager object, which manages communication with instruments
rm.list_resources()              # List all available resources (e.g., connected instruments) managed by pyvisa

In [ ]:
#Establishing Connection for NMOS tap SMU
SM_NMOS = rm.open_resource('GPIB6::20::INSTR')  # Open a connection to the instrument with the address 'ASRL3::INSTR' and assign it to SM_VIN
SM_NMOS.read_termination = '\n'             # Set the read termination character to a newline. This tells pyvisa when to consider a message as complete when reading from the instrument.
SM_NMOS.write_termination = '\n'            # Set the write termination character to a newline. This ensures that each command sent to the instrument is properly terminated.
SM_NMOS.baud_rate = 9600                    # Set the baud rate for serial communication to 9600. This is the speed at which data is transmitted over the serial connection.
print(SM_NMOS)                              # Print the resource object (SM_VIN) to display its properties
print(SM_NMOS.query('*IDN?'))               # Send the '*IDN?' command to the instrument and print its response. This command usually asks the instrument to identify itself (e.g., manufacturer, model number).

#Establishing Connection for PMOS tap SMU
SM_PMOS = rm.open_resource('GPIB6::21::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_PMOS.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_PMOS.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_PMOS.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_PMOS)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_PMOS.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument

#Establishing Connection for VDD SMU
SM_VDD = rm.open_resource('GPIB6::22::INSTR') # Open a connection to the instrument with the address 'ASRL4::INSTR' and assign it to SM_VOUT
SM_VDD.read_termination = '\n'            # Set the read termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VDD.write_termination = '\n'           # Set the write termination character to a newline for SM_VOUT, similar to SM_VIN
SM_VDD.baud_rate = 9600                   # Set the baud rate for SM_VOUT to 9600, ensuring both instruments communicate at the same speed
print(SM_VDD)                             # Print the resource object (SM_VOUT) to display its properties
print(SM_VDD.query('*IDN?'))              # Send the '*IDN?' command to the SM_VOUT instrument and print its response, identifying the instrument


#Establishing Connection for Temperature Chamber
CHAMBER = rm.open_resource('GPIB6::6::INSTR') 
CHAMBER.read_termination = '\n'            
CHAMBER.write_termination = '\n'           
CHAMBER.baud_rate = 9600                  
print(CHAMBER)                            
print(CHAMBER.query('STATUS?'))              

VDD_MAX=1.2


TEMPlist = [125, 100, 75, 50, 25, 0, -25, -50, -75, -100, -125, -150, -175]      # List of temperatures for test

<b> Initialize Source Meter (NMOS tap) <b>

In [ ]:
#TODO CONFIGURE FOR NEW TEST
SM_NMOS.write(":CONF:CURR")                 # Configure the SMU to measure current
SM_NMOS.write("OUTP:STAT 0")                # Turn off the VIN source meter
SM_NMOS.timeout = 200000                    # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
print(SM_NMOS.query(":CONF?"))              # Query and print the current configuration of the SMU
SM_NMOS.write("SOUR:VOLT:RANG 1.2")
SM_NMOS.write("SENS:CURR:PROT:LEV 0.0105")  # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_NMOS.write("SENS:CURR:RANG 1E-2")        # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_NMOS.write("CURR:NPLC 1")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.

<b> Initialize Source Meter (PMOS tap) <b>

In [ ]:
SM_PMOS.write(":CONF:CURR")                 # Configure the SMU to measure current
SM_PMOS.write("OUTP:STAT 0")                # Turn off the VIN source meter
SM_PMOS.timeout = 200000                    # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
print(SM_PMOS.query(":CONF?"))              # Query and print the current configuration of the SMU
SM_PMOS.write("SOUR:VOLT:RANG 1.2")
SM_PMOS.write("SENS:CURR:PROT:LEV 0.0105")  # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_PMOS.write("SENS:CURR:RANG 1E-2")        # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_PMOS.write("CURR:NPLC 1")                # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.

<b> Initialize Source Meter (VDD) <b>

In [ ]:
SM_VDD.write(":CONF:CURR")                  # Configure the SMU to measure current
SM_VDD.write("OUTP:STAT 0")                 # Turn off the VIN source meter
SM_VDD.timeout = 200000                     # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
print(SM_VDD.query(":CONF?"))               # Query and print the current configuration of the SMU
SM_VDD.write("SOUR:VOLT:RANG 1.2")
SM_VDD.write("SENS:CURR:PROT:LEV 0.0105")   # Set the current protection level (compliance) to 10.5 mA. This limits the maximum current to protect the device under test.
SM_VDD.write("SENS:CURR:RANG 1E-2")         # Set the current measurement range to 10 mA. This sets the expected maximum current for accurate measurement.
SM_VDD.write("CURR:NPLC 1")                 # Set the number of power line cycles (NPLC) for the current measurement to 10. 
                                            # This controls the integration time, with higher values leading to more accurate but slower measurements.

<b> Initialize Temperature Chamber <b>

In [ ]:
CHAMBER.write("ON")
CHAMBER.write("RATE=20")            # Sets the speed at which the chamber will ramp to change TEMP
CHAMBER.write("WAIT=30")            # Sets the time that the chamber will wait once the desired temperature is reached before measurements start
CHAMBER.write("HON")                # Turns heat on
CHAMBER.write("CON")                # Turns cool on
time.sleep(1) 

print(CHAMBER.query("RATE?"))
print(CHAMBER.query("WAIT?"))

<b> Enter the Device under Test <b>

In [ ]:
directory_name = input("Enter the Device under Test (DUT):")  # Prompt the user to input the temperature of the experiment, which will be used as the directory name
DUT="DUT_"+str(directory_name)
os.makedirs(DUT, exist_ok=True)                             # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True
DATE=str(DUT)+"/"+str(date.today())                         # Create another directory for test date within DUT name
os.makedirs(DATE, exist_ok=True) 

In [ ]:
#TURN ON DEVICE HERE
SM_VDD.write(":SOUR:VOLT:MODE FIX")                             # Sets source mode to fix in case sweeps are used
SM_NMOS.write(":SOUR:VOLT:MODE FIX")                    
SM_PMOS.write(":SOUR:VOLT:MODE FIX")

#TODO: figure out expected current range
#SM_VDD.write("SENS:CURR:RANG 1E-2")
#SM_NMOS.write("SENS:CURR:RANG 1E-2")                            
#SM_PMOS.write("SENS:CURR:RANG 1E-2")

SM_VDD.write("SOUR:VOLT:LEV 1.2")                               # Set VDD source value
SM_NMOS.write("SOUR:VOLT:LEV 0")  
SM_PMOS.write("SOUR:VOLT:LEV 1.2")

SM_VDD.write(":OUTP ON")                                     # Turn on the devices
#SM_NMOS.write("OUTP:STAT 1")
SM_NMOS.write(":OUTP ON")
SM_PMOS.write(":OUTP ON")

In [ ]:
# Reset both SMUs to default settings and clear errors
SM_PMOS.write("*RST; *CLS")
SM_VDD.write("*RST; *CLS")

# Clear trace buffers and triggers
SM_PMOS.write("TRAC:CLE")
SM_VDD.write("TRAC:CLE")
SM_PMOS.write("TRIG:CLE")
SM_VDD.write("TRIG:CLE")

# Configure SM_PMOS (Master)
SM_PMOS.timeout = 200000                          # Set timeout for SMU operations
SM_PMOS.write("SYST:AZER ON")                     # Enable auto-zero
SM_PMOS.write("SOUR:FUNC VOLT")                   # Set source function to voltage
SM_PMOS.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
SM_PMOS.write("SOUR:VOLT:START 0.000")            # Set start voltage
SM_PMOS.write("SOUR:VOLT:STOP 1.200")             # Set stop voltage
SM_PMOS.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size
SM_PMOS.write("SENS:FUNC 'CURR'")                 # Set measurement function to current
SM_PMOS.write("SENS:CURR:PROT 0.0105")           # Set current protection (compliance) to 50 µA
SM_PMOS.write("SENS:CURR:RANG 0.01")            # Set current range
SM_PMOS.write("SENS:CURR:NPLC 1")                 # Set NPLC for current measurement to 1
SM_PMOS.write("TRIG:COUN 121")                   # Set trigger count to match sweep points
SM_PMOS.write("TRIG:SOUR IMM")                    # Immediate trigger source for the master
SM_PMOS.write("TRIG:OUTP SOUR")                   # Set trigger output to Trigger Link
SM_PMOS.write("TRIG:OLIN 2")
SM_PMOS.write("OUTP ON")                          # Enable output

# Configure SM_VDD (Slave)
SM_VDD.timeout = 200000                           # Set timeout for SMU operations
SM_VDD.write("SYST:AZER ON")                      # Enable auto-zero
SM_VDD.write("SOUR:FUNC VOLT")                    # Set source function to voltage
SM_VDD.write("SOUR:VOLT:START 0")                # Set start voltage
SM_VDD.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
SM_VDD.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size
#SM_VDD.write("SOUR:VOLT 1.2")                     # Set source voltage
SM_VDD.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
SM_VDD.write("SENS:FUNC 'CURR'")                  # Set measurement function to current
SM_VDD.write("SENS:CURR:PROT 0.0105")            # Set current protection (compliance) to 50 µA
SM_VDD.write("SENS:CURR:RANG 0.01")             # Set current range
SM_VDD.write("SENS:CURR:NPLC 1")                  # Set NPLC for current measurement to 1
SM_VDD.write("TRIG:COUN 121")                    # Set trigger count to match sweep points
SM_VDD.write("TRIG:SOUR TLIN")                    # Trigger source is Trigger Link (slave)
SM_VDD.write("TRIG:INP SOUR")
SM_VDD.write("TRIG:ILIN 2")
SM_VDD.write("OUTP ON")                           # Enable output

# Start the measurement

SM_VDD.write("INIT")                              # Ensure SM_VDD is also ready
PMOSdata=SM_PMOS.query("READ?").split(',')                             # Initiate the sweep on SM_PMOS
VDDdata=SM_VDD.query("FETC?").split(',')
PMOSV=PMOSdata[0::5]
PMOSI=PMOSdata[1::5]
VDD=VDDdata[0::5]
IDD=VDDdata[1::5]



df = dp.DataFrame({'V PMOS':PMOSV, 'I PMOS':PMOSI, 'VDD':VDD, 'IDD':IDD})   # Create a DataFrame
#csv_path=os.path.join("test", 'VDD_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
df.to_csv('VDD_Sweep.csv', index=False)  

# Turn off outputs
SM_PMOS.write("OUTP OFF")
SM_VDD.write("OUTP OFF")

In [ ]:
# Reset both SMUs to default settings and clear errors
#SM_PMOS.write("*RST; *CLS")
#SM_VDD.write("*RST; *CLS")
SM_PMOS.write("*CLS")
SM_VDD.write("*CLS")

# Clear trace buffers and triggers
SM_PMOS.write("TRAC:CLE")
SM_VDD.write("TRAC:CLE")
SM_PMOS.write("TRIG:CLE")
SM_VDD.write("TRIG:CLE")

# Configure SM_PMOS (Master)
SM_PMOS.timeout = 200000                          # Set timeout for SMU operations
SM_PMOS.write("SYST:AZER ON")                     # Enable auto-zero
SM_PMOS.write("SOUR:FUNC VOLT")                   # Set source function to voltage 
SM_PMOS.write("SOUR:VOLT:START 0")                # Set start voltage
SM_PMOS.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
SM_PMOS.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size

SM_PMOS.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
#SM_PMOS.write(":SOUR:SWE:DIRE DOW")
SM_PMOS.write("SENS:FUNC 'CURR'")                 # Set measurement function to current
SM_PMOS.write("SENS:CURR:PROT 0.0105")           # Set current protection (compliance) to 50 µA
SM_PMOS.write("SENS:CURR:RANG 0.0001")            # Set current range
SM_PMOS.write("SENS:CURR:NPLC 1")                 # Set NPLC for current measurement to 1
SM_PMOS.write("TRIG:COUN 121")                   # Set trigger count to match sweep points
SM_PMOS.write
SM_PMOS.write("TRIG:SOUR IMM")                    # Immediate trigger source for the master
SM_PMOS.write("TRIG:OUTP SOUR")                   # Set trigger output to Trigger Link
#SM_PMOS.write("TRIG:INP SENS")
SM_PMOS.write("OUTP ON")                          # Enable output

# Configure SM_VDD (Slave)
SM_VDD.timeout = 200000                           # Set timeout for SMU operations
SM_VDD.write("SYST:AZER ON")                      # Enable auto-zero
SM_VDD.write("SOUR:FUNC VOLT")                    # Set source function to voltage
SM_VDD.write("SOUR:VOLT:START 0")                # Set start voltage
SM_VDD.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
SM_VDD.write("SOUR:VOLT:STEP 0.01")             # Set voltage step size
#SM_VDD.write("SOUR:SWE:DIRE DOW")
SM_VDD.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
SM_VDD.write("SENS:FUNC 'CURR'")                  # Set measurement function to current
SM_VDD.write("SENS:CURR:PROT 0.0105")            # Set current protection (compliance) to 50 µA
SM_VDD.write("SENS:CURR:RANG 0.0001")             # Set current range
SM_VDD.write("SENS:CURR:NPLC 1")                  # Set NPLC for current measurement to 1
SM_VDD.write("TRIG:COUN 121")                    # Set trigger count to match sweep points
SM_VDD.write("TRIG:SOUR TLIN")                    # Trigger source is Trigger Link (slave)
SM_VDD.write("TRIG:INP SOUR")                   # Set trigger output to Trigger Link
#SM_VDD.write("TRIG:OUTP SOUR")
SM_VDD.write("OUTP ON")                           # Enable output

# Start the measurement
SM_VDD.write("INIT")                              # Ensure SM_VDD is also ready
SM_PMOS.write("INIT")                             # Initiate the sweep on SM_PMOS


# Wait for the measurement to complete
SM_PMOS.write("*WAI")                             # Wait until all operations complete
SM_VDD.write("*WAI")

# Read and store current measurements from the trace buffers of both SMUs
ipmos_t = SM_PMOS.query("TRAC:DATA?")             # Retrieve data from SM_PMOS trace buffer
icc_t = SM_VDD.query("TRAC:DATA?")                # Retrieve data from SM_VDD trace buffer



# Return outputs back to normal
SM_VDD.write(":SOUR:VOLT:MODE FIX")                             # Sets source mode to fix in case sweeps are used                 
SM_PMOS.write(":SOUR:VOLT:MODE FIX")

SM_VDD.write("SOUR:VOLT:LEV 1.2")                               # Set VDD source value
SM_PMOS.write("SOUR:VOLT:LEV 1.2")

SM_VDD.write("OUTP:STAT 1")                                     # Turn on the devices
SM_PMOS.write("OUTP:STAT 1")

<b> Test Protocol </b>

In [ ]:
#TURN ON DEVICE HERE
SM_VDD.write(":SOUR:VOLT:MODE FIX")                             # Sets source mode to fix in case sweeps are used
SM_NMOS.write(":SOUR:VOLT:MODE FIX")                    
SM_PMOS.write(":SOUR:VOLT:MODE FIX")

#TODO: figure out expected current range
#SM_VDD.write("SENS:CURR:RANG 1E-2")
#SM_NMOS.write("SENS:CURR:RANG 1E-2")                            
#SM_PMOS.write("SENS:CURR:RANG 1E-2")

SM_VDD.write("SOUR:VOLT:LEV 1.2")                               # Set VDD source value
SM_NMOS.write("SOUR:VOLT:LEV 0")  
SM_PMOS.write("SOUR:VOLT:LEV 1.2")

SM_VDD.write("OUTP:STAT 1")                                     # Turn on the devices
#SM_NMOS.write("OUTP:STAT 1")
SM_NMOS.write(":OUTP ON")
SM_PMOS.write("OUTP:STAT 1")



for TEMP in TEMPlist:
    local=str(DATE)+"/TEMP_"+str(TEMP)                           # The temperature of the experiment gets set as the sub directory name
    os.makedirs(local, exist_ok=True)                           # Create a directory with the given name. If the directory already exists, it won't raise an error due to exist_ok=True
    
    CHAMBER.write("WAIT=30")                                    # Resets the wait time of the temp chamber to 30 minutes so it doesn't time out mid test
    CHAMBER.write("SET=" + str(TEMP))
    # TODO: There is probably a better way to check for temperature settling and do wait time
    while (CHAMBER.query("STATUS?")[3] == 'N'):                 # Checks if the temperature chamber has arrived at TEMP yet or not
        time.sleep(15)
        print(CHAMBER.query("STATUS?"))  
    print(CHAMBER.query("STATUS?"))
    time.sleep(300)                                              # Soak time

    #####################################################
    ##################### VDD Sweep #####################
    #####################################################
    # Reset both SMUs to default settings and clear errors
    #SM_PMOS.write("*RST; *CLS")
    #SM_VDD.write("*RST; *CLS")
    SM_PMOS.write("*CLS")
    SM_VDD.write("*CLS")

    # Clear trace buffers and triggers
    SM_PMOS.write("TRAC:CLE")
    SM_VDD.write("TRAC:CLE")
    SM_PMOS.write("TRIG:CLE")
    SM_VDD.write("TRIG:CLE")

    # Configure SM_PMOS (Master)
    SM_PMOS.timeout = 200000                          # Set timeout for SMU operations
    SM_PMOS.write("SYST:AZER ON")                     # Enable auto-zero
    SM_PMOS.write("SOUR:FUNC VOLT")                   # Set source function to voltage 
    SM_PMOS.write("SOUR:VOLT:START 0")                # Set start voltage
    SM_PMOS.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
    SM_PMOS.write("SOUR:VOLT:STEP 0.001")             # Set voltage step size
    SM_PMOS.write("SOUR:SWE:DIRE DOW")
    SM_PMOS.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
    SM_PMOS.write("SENS:FUNC 'CURR'")                 # Set measurement function to current
    SM_PMOS.write("SENS:CURR:PROT 0.00005")           # Set current protection (compliance) to 50 µA
    SM_PMOS.write("SENS:CURR:RANG 0.0001")            # Set current range
    SM_PMOS.write("SENS:CURR:NPLC 1")                 # Set NPLC for current measurement to 1
    SM_PMOS.write("TRIG:COUN 1201")                   # Set trigger count to match sweep points
    SM_PMOS.write("TRIG:SOUR IMM")                    # Immediate trigger source for the master
    SM_PMOS.write("TRIG:OUTP TLIN")                   # Set trigger output to Trigger Link
    SM_PMOS.write("OUTP ON")                          # Enable output

    # Configure SM_VDD (Slave)
    SM_VDD.timeout = 200000                           # Set timeout for SMU operations
    SM_VDD.write("SYST:AZER ON")                      # Enable auto-zero
    SM_VDD.write("SOUR:FUNC VOLT")                    # Set source function to voltage
    SM_VDD.write("SOUR:VOLT:START 0")                # Set start voltage
    SM_VDD.write("SOUR:VOLT:STOP 1.2")                   # Set stop voltage
    SM_VDD.write("SOUR:VOLT:STEP 0.001")             # Set voltage step size
    SM_VDD.write("SOUR:SWE:DIRE DOW")
    SM_VDD.write("SOUR:VOLT:MODE SWE")               # Set source mode to sweep
    SM_VDD.write("SENS:FUNC 'CURR'")                  # Set measurement function to current
    SM_VDD.write("SENS:CURR:PROT 0.00005")            # Set current protection (compliance) to 50 µA
    SM_VDD.write("SENS:CURR:RANG 0.0001")             # Set current range
    SM_VDD.write("SENS:CURR:NPLC 1")                  # Set NPLC for current measurement to 1
    SM_VDD.write("TRIG:COUN 1201")                    # Set trigger count to match sweep points
    SM_VDD.write("TRIG:SOUR TLIN")                    # Trigger source is Trigger Link (slave)
    SM_VDD.write("OUTP ON")                           # Enable output

    # Start the measurement
    SM_PMOS.write("INIT")                             # Initiate the sweep on SM_PMOS
    SM_VDD.write("INIT")                              # Ensure SM_VDD is also ready

    # Wait for the measurement to complete
    SM_PMOS.write("*WAI")                             # Wait until all operations complete
    SM_VDD.write("*WAI")

    # Read and store current measurements from the trace buffers of both SMUs
    ipmos_t = SM_PMOS.query("TRAC:DATA?")             # Retrieve data from SM_PMOS trace buffer
    icc_t = SM_VDD.query("TRAC:DATA?")                # Retrieve data from SM_VDD trace buffer

  

    # Return outputs back to normal
    SM_VDD.write(":SOUR:VOLT:MODE FIX")                             # Sets source mode to fix in case sweeps are used                 
    SM_PMOS.write(":SOUR:VOLT:MODE FIX")

    SM_VDD.write("SOUR:VOLT:LEV 1.2")                               # Set VDD source value
    SM_PMOS.write("SOUR:VOLT:LEV 1.2")

    SM_VDD.write("OUTP:STAT 1")                                     # Turn on the devices
    SM_PMOS.write("OUTP:STAT 1")

    ######################################################
    ##################### NMOS Sweep #####################
    ######################################################
    SM_NMOS.write("*CLS")
    SM_NMOS.timeout = 200000                  # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    # Clear trace buffers and triggers
    SM_NMOS.write("TRAC:CLE")
    SM_NMOS.write("TRIG:CLE")

    SM_NMOS.write(":SOUR:VOLT:STAR 0")
    SM_NMOS.write(":SOUR:VOLT:STOP 1.2")
    SM_NMOS.write(":SOUR:VOLT:STEP .001")

    SM_NMOS.write(":SOUR:VOLT:MODE SWE")
    SM_NMOS.write(":SOUR:SWE:SPAC LIN")
    SM_NMOS.write(":SOUR:SWE:RANG AUTO")      # Set the sweep range to auto, allowing the SMU to automatically adjust the range.
    SM_NMOS.write(":SOUR:SWE:SPAC LIN")       # Set the sweep spacing to linear, meaning the voltage will increase linearly between steps.
    SM_NMOS.write(":SOUR:SWE:CAB EARL")       # Set the sweep to begin before the trigger event (EARL mode).

    SM_NMOS.write(":TRIG:COUN 1201")
    SM_NMOS.write(":SOUR:DEL 0.001")

    # Start the measurement
    NMOSdata = SM_NMOS.query(":READ?").split(',')
    NMOSV=NMOSdata[0::5]
    NMOSI=NMOSdata[1::5]

    df = dp.DataFrame({'V NMOS':NMOSV, 'I NMOS':NMOSI})   # Create a DataFrame
    csv_path=os.path.join(DATE, 'NMOS_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
    df.to_csv(csv_path, index=False)    
    SM_NMOS.write(":SOUR:VOLT:MODE:FIX ")
    SM_NMOS.write("SOUR:VOLT:LEV 0") 
    
    ######################################################
    ##################### PMOS Sweep #####################
    ######################################################
    SM_PMOS.write("*CLS")
    SM_PMOS.timeout = 200000                  # Set the timeout for the SMU operations to 200,000 milliseconds (200 seconds).
    # Clear trace buffers and triggers
    SM_PMOS.write("TRAC:CLE")
    SM_PMOS.write("TRIG:CLE")

    SM_PMOS.write(":SOUR:VOLT:STAR 0")
    SM_PMOS.write(":SOUR:VOLT:STOP 1.2")
    SM_PMOS.write(":SOUR:VOLT:STEP .001")
    SM_PMOS.write(":SOUR:SWE:DIRE DOW")

    SM_PMOS.write(":SOUR:VOLT:MODE SWE")
    SM_PMOS.write(":SOUR:SWE:SPAC LIN")
    SM_PMOS.write(":SOUR:SWE:RANG AUTO")      # Set the sweep range to auto, allowing the SMU to automatically adjust the range.
    SM_PMOS.write(":SOUR:SWE:SPAC LIN")       # Set the sweep spacing to linear, meaning the voltage will increase linearly between steps.
    SM_PMOS.write(":SOUR:SWE:CAB EARL")       # Set the sweep to begin before the trigger event (EARL mode).

    SM_PMOS.write(":TRIG:COUN 1201")
    SM_PMOS.write(":SOUR:DEL 0.001")

    # Start the measurement
    PMOSdata = SM_PMOS.query(":READ?").split(',')
    PMOSV=PMOSdata[0::5]
    PMOSI=PMOSdata[1::5]

    df = dp.DataFrame({'V PMOS':PMOSV, 'I PMOS':PMOSI})   # Create a DataFrame
    csv_path=os.path.join(DATE, 'PMOS_Sweep.csv')                                             # Define the file path for saving the CSV file in the specified directory.
    df.to_csv(csv_path, index=False)    
    SM_PMOS.write(":SOUR:VOLT:MODE:FIX ")
    SM_PMOS.write("SOUR:VOLT:LEV 1.2")    

    

###############################################################
##################### Shut Down Procedure #####################
###############################################################
SM_PMOS.write("OUTP:STAT 0")                                  # Turn off the source meters
SM_NMOS.write("OUTP:STAT 0")                              
SM_VDD.write("OUTP:STAT 0")                                
CHAMBER.write("SET=30")                             # Sets the temperature back to room temp once test is complete                      
while (CHAMBER.query("STATUS?")[3] == 'N'):         # Checks if chamber has arrived at room temp
    time.sleep(15)
CHAMBER.write("STOP")                               # Once chamber is at room temp, chamber stops operation

In [ ]:
SM_PMOS.write("OUTP:STAT 0")                                 # Turn off the VOUT source meter
SM_NMOS.write("OUTP:STAT 0") 
SM_VDD.write("OUTP:STAT 0")

In [ ]:
#CHAMBER.write("SET=25")
CHAMBER.write("STOP")